In [4]:
import cirq

In [5]:
import mitiq

In [17]:
from cirq import LineQubit, Circuit, H,CNOT, measure_each
circuit = cirq.Circuit()
qreg = cirq.LineQubit.range(2)
# circuit.append(cirq.H(q0))
# circuit.append(cirq.CNOT(q0,q1))
circuit = Circuit(H(qreg[0]),CNOT(qreg[0],qreg[1]), measure_each(*qreg))
print(circuit)

0: ───H───@───M───
          │
1: ───────X───M───


In [32]:
from functools import partial

import numpy as np
from cirq.experiments.single_qubit_readout_calibration_test import (
    NoisySingleQubitReadoutSampler,
)

from mitiq import MeasurementResult

def noisy_readout_executor(
    circuit, p0: float = 0.01, p1: float = 0.01, shots: int = 1000
) -> MeasurementResult:
    # Replace with code based on your frontend and backend.
    simulator = NoisySingleQubitReadoutSampler(p0, p1)
    result = simulator.run(circuit, repetitions=shots)

    return MeasurementResult(
        result=np.column_stack(list(result.measurements.values())),
        qubit_indices=tuple(
            # q[2:-1] is necessary to convert "q(number)" into "number"
            int(q[2:-1])
            for k in result.measurements.keys()
            for q in k.split(",")
        ),
    )

# Use a noisy executor that has a 25% chance of flipping
p_flip = 0.25
noisy_executor = partial(noisy_readout_executor, p0=p_flip, p1=p_flip)

In [33]:
from mitiq.executor.executor import Executor

executor = Executor(noisy_executor)
result = executor.run([circuit])
noisy_result = result[0]
print(noisy_result.get_counts())

{'10': 171, '11': 319, '00': 312, '01': 198}


In [36]:
from mitiq.rem import generate_inverse_confusion_matrix

inverse_confusion_matrix = generate_inverse_confusion_matrix(2, p_flip, p_flip)
print(inverse_confusion_matrix) #shows the inversion matrix(A) ,when we perform the inverse of A matrix on the noisy result we get the near to ideal result.

[[ 2.25 -0.75 -0.75  0.25]
 [-0.75  2.25  0.25 -0.75]
 [-0.75  0.25  2.25 -0.75]
 [ 0.25 -0.75 -0.75  2.25]]


In [35]:
from mitiq.rem.inverse_confusion_matrix import mitigate_measurements

mitigated_result = mitigate_measurements(noisy_result, inverse_confusion_matrix)
print(mitigated_result.get_counts())

{'11': 515, '00': 483, '01': 2}
